In [16]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep

In [17]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [18]:
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_NAME')
kafka_topic_partition = os.getenv('KAFKA_TOPIC_NAME')

In [19]:
print(kafka_host)
print(kafka_topic)
print(kafka_topic_partition)
print(dotenv_path)
print(os.environ.get('KAFKA_TOPIC_NAME'))

dataeng-kafka
apotek-dibimbing-ajinusa
apotek-dibimbing-ajinusa
/resources/.env
apotek-dibimbing-ajinusa


In [20]:
producer = KafkaProducer(bootstrap_servers=f'{kafka_host}:9092')

In [21]:
_instance = Faker()
global faker
faker = Faker()

In [22]:
from datetime import datetime
# DataGenerator untuk penjualan obat
class DataGenerator(object):
    @staticmethod
    def get_data():
        transaction_date = faker.date_this_year()  # Mendapatkan tanggal transaksi
        transaction_time = faker.time()  # Mendapatkan waktu transaksi

        # Mengonversi tanggal dan waktu menjadi string dalam format yang dapat diterima JSON
        transaction_date_str = transaction_date.strftime('%Y-%m-%d') if isinstance(transaction_date, datetime) else str(transaction_date)
        transaction_time_str = transaction_time  # Faker time() sudah berupa string (jam:menit:detik)
        
         # Data transaksi
        quantity = faker.random_int(min=1, max=5)  # Jumlah obat yang dibeli
        unit_price = faker.random_int(min=1000, max=20000)  # Harga per unit obat (dalam IDR)
        total_price = quantity * unit_price  # Total harga transaksi

        return [
            uuid.uuid4().__str__(),  # Transaction ID
            faker.name(),  # Nama pembeli
            faker.random_element(elements=('Paracetamol', 'Ibuprofen', 'Amoxicillin', 'Vitamin C', 'Antibiotik', 'Cough Syrup')),  # Nama obat
            quantity,  # Jumlah obat yang dibeli
            unit_price,  # Harga per unit obat (dalam IDR)
            total_price,  # Total harga transaksi
            faker.random_element(elements=('Cash', 'Credit Card', 'Debit Card', 'QRIS')),  # Metode pembayaran
            transaction_date_str,  # Tanggal transaksi sebagai string
            transaction_time_str  # Waktu transaksi sebagai string
        ]

# Mengirimkan data ke Kafka
for i in range(1, 200):  # Jumlah data yang akan dibuat
    columns = ["transaction_id", "buyer_name", "medication_name", "quantity", "unit_price", "total_price", "payment_method", "transaction_date", "transaction_time"]
    data_list = DataGenerator.get_data()
    json_data = dict(zip(columns, data_list))
    _payload = json.dumps(json_data).encode("utf-8")
    response = producer.send(topic=kafka_topic_partition, value=_payload)
    print(json_data['transaction_id'], response.get())  # Menampilkan ID transaksi dan status pengiriman
    sleep(5)  # Delay 5 detik antara setiap pengiriman


b8dd9554-0d80-45b0-920a-3580fe72b509 RecordMetadata(topic='apotek-dibimbing-ajinusa', partition=0, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa', partition=0), offset=416, timestamp=1737526887609, log_start_offset=379, checksum=None, serialized_key_size=-1, serialized_value_size=273, serialized_header_size=-1)
84b9be8d-1bcd-4674-b0c1-6264b453b9c3 RecordMetadata(topic='apotek-dibimbing-ajinusa', partition=1, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa', partition=1), offset=383, timestamp=1737526892621, log_start_offset=351, checksum=None, serialized_key_size=-1, serialized_value_size=278, serialized_header_size=-1)
c9ec3ab9-12c1-4e9e-abbc-e3aa7316f8aa RecordMetadata(topic='apotek-dibimbing-ajinusa', partition=4, topic_partition=TopicPartition(topic='apotek-dibimbing-ajinusa', partition=4), offset=368, timestamp=1737526897629, log_start_offset=323, checksum=None, serialized_key_size=-1, serialized_value_size=275, serialized_header_size=-1)
ab599bdc-e